In [1]:
from __future__ import absolute_import, division, print_function

import sqlite3
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import utils.pandas_utils as pd_utils
import utils.units_utils as units
%load_ext autoreload
%autoreload 2

# Read in trial_lc.db into Pandas df
db = sqlite3.connect('trial_lc.db')

cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
cursor.close()

df = {}
for table in ['agn_lc', 'obs_metadata', 'sprinkled_objects',]:
    df[table] =  pd.read_sql_query("SELECT * from %s" %table, db)
    #df[table].to_csv('%s.csv' %table, index_label='index')

db.close()

# Read in truth.sqlite into Pandas df

truth = sqlite3.connect('truth.sqlite')
#truth_cursor = truth.cursor()
#truth_cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
#print(truth_cursor.fetchall())
#truth_cursor.close()
df['zpoint'] = pd.read_sql_query("SELECT * from 'zpoint'", truth)
truth.close()

#####################
# Preprocess zpoint #
#####################

# Unravel the dictionary inside the dictionary, of DRW parameters
df['zpoint'] = pd_utils.dict_of_dict_to_columns(df['zpoint'], dict_column='varParamStr', dict_key='p')

# Keep only the columns we care about
keep_truth_cols = ['uniqueId', 'galaxy_id', 'redshift', 'magNorm', 'agn_tau', 't0Delay']
keep_truth_cols += ['agn_sf%s' %b for b in 'ugrizy']
df['zpoint'] = df['zpoint'][keep_truth_cols]

# Keep only the rows with t0Delay values that are not NaN
df['zpoint'] = df['zpoint'][df['zpoint']['t0Delay'].notnull()]

# Rename some columns to avoid conflict later
df['zpoint'].rename(columns={'redshift': 'truth_redshift', 'galaxy_id': 'truth_galaxy_id'}, inplace=True)

#print(df['zpoint'].shape, df['agn_lc'].shape, df['obs_metadata'].shape)

#print(df['zpoint'].columns, df['agn_lc'].columns, df['obs_metadata'].columns)

#############
# Pre-check #
#############

# Check that the uniqueIds in sprinkled_objects are the same as those in agn_lc
assert pd_utils.a_relationship_b(a=df['sprinkled_objects'], b=df['agn_lc'], column='uniqueId', relationship='equal')

# Check that the uniqueIds in sprinkled_objects are same as those in zpoint
assert pd_utils.a_relationship_b(a=df['sprinkled_objects'], b=df['zpoint'], column='uniqueId', relationship='equal')

# Check that the obshistids in agn_lc are in obs_metadata
assert pd_utils.a_relationship_b(a=df['agn_lc'], b=df['obs_metadata'], column='obshistid', relationship='subset')

# Check if number of observations for each light curve is even
print(df['agn_lc'].groupby('uniqueId')['obshistid'].nunique())

# Check that the lens galaxies in sprinkled_objects are in zpoint
# Note: this will fail due to a bug in the zpoint table where the uniqueId-galaxy_id pairings are wrong.
#assert pd_utils.a_subset_of_b(a=df['sprinkled_objects'], b=df['zpoint'], 
#                              column_a='galaxy_id', column_b='truth_galaxy_id')

######################
# Remove unnecessary #
######################
# Warning: Original dataframes are altered.

# Remove obshistids that aren't in agn_lc, from obs_metadata
lc_obshistids = df['agn_lc']['obshistid'].unique().copy()
mask_obs = df['obs_metadata']['obshistid'].isin(lc_obshistids)
df['obs_metadata'] = df['obs_metadata'].loc[mask_obs]

# Total number of observed visits should equal the number of agn images times number of visits per agn image 
lc_uniqueIds = df['agn_lc']['uniqueId'].unique().copy()
assert df['agn_lc'].shape[0] == df['obs_metadata'].shape[0]*len(lc_uniqueIds)

# Merge agn_lc and sprinkled_objects
sprinkled_lc = pd.merge(df['agn_lc'], df['sprinkled_objects'], on='uniqueId')

# Merge this with zpoint
sprinkled_lc_withtruth = pd.merge(sprinkled_lc, df['zpoint'], on='uniqueId')

# Merge this with obs_metadata
sprinkled_lc_obs_withtruth = pd.merge(sprinkled_lc_withtruth, df['obs_metadata'], on='obshistid')

# Check shape
assert np.array_equal(sprinkled_lc_obs_withtruth.shape,
                      [len(lc_uniqueIds)*len(lc_obshistids),
                       df['agn_lc'].shape[1] + df['sprinkled_objects'].shape[1] + df['zpoint'].shape[1] + df['obs_metadata'].shape[1] - 3])

# Derive 1-sigma depth from 5-sigma depth
sprinkled_lc_obs_withtruth['mag_err'] = units.scale_mag_as_flux(sprinkled_lc_obs_withtruth['fiveSigmaDepth'], flux_scale=0.2)

sprinkled_lc_obs_withtruth.to_csv("joined_sprinkled.csv", index=False)

In [17]:
list(sprinkled_lc_obs_withtruth.columns)

['uniqueId',
 'obshistid',
 'mag',
 'galaxy_id',
 'ra',
 'dec',
 'redshift',
 'truth_galaxy_id',
 'truth_redshift',
 'magNorm',
 'agn_tau',
 't0Delay',
 'agn_sfu',
 'agn_sfg',
 'agn_sfr',
 'agn_sfi',
 'agn_sfz',
 'agn_sfy',
 'mjd',
 'filter',
 'fiveSigmaDepth',
 'mag_err']